In [ ]:
import re
import json
from pathlib import Path
import json
from label import tag_sentence

In [ ]:
input_file = Path("../data/braces-data/braces-Schleswig-Holstein-2023-06-07.txt")
output_file = Path("../data/sorted_citations_cleaned_fixed.jsonl")

citation_clean_pattern = re.compile(r"^(?:\.\./[^ ]*|vgl\.|https?://\S+)\s*")
ecli_prefix = "../../gesp-experiments/unified-gesp-dataset/"

records = []

with input_file.open("r", encoding="utf-8", errors="replace") as f:
    for line in f:
        line = line.strip()
        if line.startswith(ecli_prefix):
            parts = line.split(" ", 1)
            if len(parts) == 2:
                path_part, citation_part = parts
                ecli = path_part.split("/")[-1]
                cleaned_citation = re.sub(citation_clean_pattern, '', citation_part)
                records.append({
                    "ecli": ecli,
                    "citation": cleaned_citation
                })

with output_file.open("w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"Done {len(records)} citations saved to {output_file}")


In [ ]:
input_path = "../data/sorted_citations_cleaned_fixed.jsonl"
output_path = "../data/tag_only_classification.jsonl"

results = []

with open(input_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        item = json.loads(line)
        citation = item.get("citation", "")
        ecli = item.get("ecli", "")
        
        tags = tag_sentence(citation)
        label = "CITATION" if any(tag["tag"] != "O" for tag in tags) else "NOT_CITATION"
        
        results.append({
            "ecli": ecli,
            "citation": citation,
            "label": label,
            "tags": tags
        })

with open(output_path, "w", encoding="utf-8") as outfile:
    for entry in results:
        outfile.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Tag-only classification complete — {len(results)} processed and saved to {output_path}")
